In [52]:
%load_ext autoreload
%autoreload 2
from social_distancing import PersonAgent, SocialDistanceModel
import altair as alt
import pandas as pd
import plotly.express as px

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [109]:
%%time

model = SocialDistanceModel(200, 800, 400, social_distancing_p=.8)

for _ in range(300):
    model.step()

Wall time: 681 ms


In [110]:
def unpack_position(pos):
    return pd.Series({"x" : pos[0], "y" : pos[1]})

In [111]:
model_data = model.datacollector.get_model_vars_dataframe().reset_index().rename(columns={'index' : "Step"})

model_data_melted = pd.melt(model_data, id_vars="Step", var_name="State", value_name="Count")

In [112]:
model_data_melted['norm'] = model_data_melted['Count'] / 200

model_data_melted.head()

,Step,State,Count,norm
0,0,n_healthy,199,0.995
1,1,n_healthy,199,0.995
2,2,n_healthy,199,0.995
3,3,n_healthy,199,0.995
4,4,n_healthy,199,0.995


In [113]:
model_data_melted['sort_order'] = model_data_melted['State'].map({'n_sick' : 0, 'n_healthy' : 1, 'n_recovered' : 2})
model_data_melted = model_data_melted.sort_values(["Step", "sort_order"])

In [114]:
model_data_melted

,Step,State,Count,norm,sort_order
300,0,n_sick,1,0.005,0
0,0,n_healthy,199,0.995,1
600,0,n_recovered,0,0.000,2
301,1,n_sick,1,0.005,0
1,1,n_healthy,199,0.995,1
...,...,...,...,...,...
298,298,n_healthy,52,0.260,1
898,298,n_recovered,110,0.550,2
599,299,n_sick,39,0.195,0
299,299,n_healthy,51,0.255,1


In [115]:
c = (
    alt.Chart(model_data_melted)
    .mark_area()
    .encode(
        x="Step",
        y=alt.Y("Count:Q", stack="zero"),
        color=alt.Color("State:N", sort=["n_sick", "n_healthy", "n_recovered"]),
        order=alt.Order("sort_order"),
    )
)
c

alt.Chart(...)

In [116]:
agent_positions = model.datacollector.get_agent_vars_dataframe()

position_split_df = agent_positions['pos'].apply(unpack_position)

agent_df = agent_positions.join(position_split_df).drop(['pos'], axis='columns').reset_index()

In [117]:
dummy_recovery_agent = pd.DataFrame([{'Step' : 0, 'AgentID' : -1, 'state' : 'recovered', 'x' : 0, 'y' : 0}])

In [118]:
agent_df_recovery = dummy_recovery_agent.append(agent_df).reset_index(drop=True)

In [119]:
agent_df_recovery

,Step,AgentID,state,x,y
0,0,-1,recovered,0.000000,0.000000
1,0,0,sick,284.399025,241.931137
2,0,1,healthy,582.716644,83.245239
3,0,2,healthy,666.374869,228.006874
4,0,3,healthy,666.330060,256.483175
...,...,...,...,...,...
59996,299,195,healthy,322.484237,181.098626
59997,299,196,healthy,280.293528,121.477744
59998,299,197,recovered,571.774558,101.665310
59999,299,198,recovered,198.826782,264.843731


In [120]:
fig = px.scatter(
    agent_df,
    x="x",
    y="y",
    size=[10] * len(agent_df),
    size_max=10,
    animation_frame="Step",
    animation_group="AgentID",
    hover_name="AgentID",
    color="state",
    range_x=[0, 800],
    range_y=[400, 0],
    color_discrete_map={"healthy": "green", "sick": "red", "recovered": "blue"},
)
# fig.show()
fig.write_html("animation-state.html")

In [121]:
len(agent_df)

60000

In [38]:
agent_df.query("159 < Step < 164").head(100)

,Step,AgentID,state,x,y
32000,160,0,recovered,415.084648,395.748626
32001,160,1,sick,413.203987,126.516686
32002,160,2,sick,124.552326,145.134431
32003,160,3,sick,295.976715,272.673348
32004,160,4,recovered,236.012671,372.780316
32005,160,5,sick,417.950441,263.172656
32006,160,6,sick,12.993374,33.885872
32007,160,7,recovered,93.543556,395.090655
32008,160,8,sick,516.574892,298.242780
32009,160,9,sick,138.642125,89.791059


In [37]:
pd.set_option("display.max_rows", 250)

In [44]:
subset_steps = agent_df.query("150 < Step < 180")

fig = px.scatter(
    subset_steps,
    x="x",
    y="y",
    size=[10] * len(subset_steps),
    size_max=10,
    animation_frame="Step",
    animation_group="AgentID",
    hover_name="AgentID",
    color="state",
    range_x=[0, 800],
    range_y=[400, 0],
    color_discrete_map={"healthy": "green", "sick": "red", "recovered": "blue"},
)
fig.write_html("animation-state-test.html")